## Soild classifier train

In [123]:
import torch
import torchvision
import pathlib
import glob
from torchvision.transforms import transforms

In [124]:
class ConvNet(torch.nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        # self.batchSize = batch_size

        # ((inputsize-kernel+2*Padding)/stride) +1
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=12, kernel_size=(3,3), stride=1, padding=1),
            torch.nn.BatchNorm2d(num_features=12),
            torch.nn.ReLU(),
        )

        self.maxpool1 = torch.nn.MaxPool2d(kernel_size=(2,2))


        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=12, out_channels=20, kernel_size=(3,3), stride=1, padding=1),
            torch.nn.BatchNorm2d(num_features=20),
            torch.nn.ReLU(),
        )

        self.conv3 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=20, out_channels=30, kernel_size=(3,3), stride=1, padding=1),
            # output: 30,20,20
            torch.nn.BatchNorm2d(num_features=30),
            torch.nn.ReLU(),
        )

        self.maxpool2 = torch.nn.MaxPool2d(kernel_size=(2,2))


        # Flatten the output before fully connected layers
        self.flatten = torch.nn.Flatten()

        # Fully connected layers
        self.fc1 = torch.nn.Linear(in_features=30* 37* 37, out_features=128)
        self.relu4 = torch.nn.ReLU()

        self.fc2 = torch.nn.Linear(in_features=128, out_features=num_classes)

    def forward(self, x):
        x = self.conv1(x)
        # print("conv1",x.size())

        x = self.maxpool1(x)
        # print("maxpool1",x.size())

        x = self.conv2(x)
        # print("conv2",x.size())

        x = self.conv3(x)
        # print("conv3",x.size())

        x = self.maxpool2(x)
        # print("maxpool2",x.size())

        # Flatten the output
        x = self.flatten(x)
        # print("flatten",x.size())

        # Fully connected layers
        x = self.fc1(x)
        # print("fc1",x.size())

        x = self.relu4(x)
        # print("relu4",x.size())

        x = self.fc2(x)
        # print("fc2",x.size())


        return x


In [125]:
# Get cpu, gpu or mps device for training.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [126]:
model = ConvNet(num_classes=3).to(device=device)

In [127]:
print(model)

ConvNet(
  (conv1): Sequential(
    (0): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Sequential(
    (0): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv3): Sequential(
    (0): Conv2d(20, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=41070, out_features=128, bias=True)
  (relu4): ReLU()
  (fc2): Linear(in_features=128, out_features

In [128]:
transformer = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(+90),
    transforms.ToTensor(),  # 0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5, 0.5, 0.5],  # 0-1 to [-1,1] , formula (x-mean)/std
                         [0.5, 0.5, 0.5])
])

In [129]:
train_path = './data/train'
val_path = './data/validation'

In [130]:
batchSize = 4
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform=transformer),
    batch_size=batchSize, shuffle=True
)
val_loader = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(val_path, transform=transformer),
    batch_size=batchSize//2, shuffle=True
)

In [131]:
# categories
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

# CNN Network
print(classes)

['Gravel', 'Sand', 'Silt']


In [132]:
model = ConvNet(num_classes=len(classes)).to(device)

In [133]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [134]:
# calculating the size of training and testing images
train_count = len(glob.glob(train_path + '/**/*.jpg'))
test_count = len(glob.glob(val_path + '/**/*.jpg'))

In [135]:
print(train_count, test_count)


147 147


In [138]:
for epoch in range(5):
    # Training
    model.train()
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)

        output = model(X)
        loss = torch.nn.functional.cross_entropy(output, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print(epoch, loss.item())

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0

        for X_val, y_val in val_loader:
            X_val, y_val = X_val.to(device), y_val.to(device)

            val_output = model(X_val)
            val_loss += torch.nn.functional.cross_entropy(val_output, y_val).item()

            _, predicted = val_output.max(1)
            total += y_val.size(0)
            correct += predicted.eq(y_val).sum().item()

        average_val_loss = val_loss / len(val_loader)
        accuracy = correct / total

        print(f"Epoch {epoch} - Validation Loss: {average_val_loss:.4f}, Accuracy: {100 * accuracy:.2f}%")

0 0.5116536021232605
0 0.04820380359888077
0 0.30285438895225525
0 0.778286874294281
0 0.18381761014461517
0 0.08878567069768906
0 0.7207648754119873
0 0.5898793935775757
0 3.0445287227630615
0 1.8412678241729736
0 0.026245875284075737
0 0.2089916467666626
0 0.23708800971508026
0 0.336184024810791
0 0.0739688128232956
0 0.009684115648269653
0 0.14372915029525757
0 3.2998499870300293
0 0.10084935277700424
0 0.26736047863960266
0 0.24864265322685242
0 0.05333418399095535
0 0.26546990871429443
0 0.0004816311993636191
0 0.34140169620513916
0 0.371107280254364
0 0.15757186710834503
0 0.03291301429271698
0 0.013061122968792915
0 1.512782335281372
0 0.2963249683380127
0 0.5363077521324158
0 0.7796669006347656
0 0.021962717175483704
0 0.29563775658607483
0 0.16298474371433258
0 0.46173444390296936
0 0.33888331055641174
Epoch 0 - Validation Loss: 0.4430, Accuracy: 82.67%
1 0.19365835189819336
1 1.794156789779663
1 0.7348855137825012
1 0.42204946279525757
1 0.11232858896255493
1 0.04230498149991

In [141]:
test_path = './data/test'
test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform=transformer),
    batch_size=batchSize, shuffle=True
)

model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    test_loss = 0.0
    correct = 0
    total = 0

    for X_test, y_test in test_loader:
        X_test, y_test = X_test.to(device), y_test.to(device)

        test_output = model(X_test)
        test_loss += torch.nn.functional.cross_entropy(test_output, y_test).item()

        _, predicted = test_output.max(1)
        total += y_test.size(0)
        correct += predicted.eq(y_test).sum().item()

    average_test_loss = test_loss / len(test_loader)
    accuracy = correct / total

    print(f"Test Loss: {average_test_loss:.4f}, Accuracy: {100 * accuracy:.2f}%")

Test Loss: 0.4772, Accuracy: 83.33%
